<a href="https://colab.research.google.com/github/y001003/Bootcamp/blob/main/ai06_%EB%B0%95%EC%98%81%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DS SC23x 
## Applied Predictive Modeling

 이번 스프린트 챌린지에서는 시카고에 있는 식당들의 정보와 해당 식당들의 위생 검사 결과에 관한 데이터를 담은 데이터셋을 다루게 됩니다.

데이터셋에 관한 설명은 이 [PDF](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF)를 참고해주시길 바랍니다.

#### 오늘 여러분은 Chicago시의 공중보건부가 진행한 위생 검사의 실패 여부를 예측하는 모델을 만들어야 합니다.

여러분의 모델이 예측할 target은 `Inspection Fail` 칼럼입니다.   
칼럼 값은 아래와 같습니다:
- 식당이 위생 검사에 불합격한 경우: **1**
- 식당이 검사를 통과한 경우: **0**

In [ ]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

In [ ]:
# 데이터셋을 불러오기 위해 판다스 라이브러리를 불러옵니다
import pandas as pd

train_url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/food_inspection_sc23x/food_ins_train.csv'
test_url  = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/food_inspection_sc23x/food_ins_test.csv'

# train, test 데이터셋을 불러옵니다
train = pd.read_csv(train_url)
test  = pd.read_csv(test_url)

# 데이터셋 확인
assert train.shape == (60000, 17)
assert test.shape  == (20000, 17)

# Part 1: 데이터 전처리 (Data Preprocessing)

In [ ]:
## 1.1 데이터셋을 파악하기 위한 EDA를 진행하세요
> EDA를 하는 방식 및 라이브러리에 대한 제한은 없습니다. 단, 시간 분배에 주의하세요.

In [ ]:
train

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Violations,Latitude,Longitude,Location,Inspection Fail
0,2050629,MY SWEET STATION INC,MY SWEET STATION,2327223.0,Restaurant,Risk 1 (High),2511 N LINCOLN AVE,CHICAGO,IL,60614.0,2017-05-18,Canvass,NaN,41.927577,-87.651528,"(-87.65152817242594, 41.92757677830966)",0
1,2078428,OUTTAKES,RED MANGO,2125004.0,Restaurant,Risk 2 (Medium),10 S DEARBORN ST FL,CHICAGO,IL,60603.0,2017-08-14,Canvass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.881807,-87.629543,"(-87.62954311539407, 41.88180696006542)",0
2,1591748,JAFFA BAGELS,JAFFA BAGELS,2278918.0,Restaurant,Risk 1 (High),225 N MICHIGAN AVE,CHICAGO,IL,60601.0,2015-12-15,Complaint,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.886377,-87.624382,"(-87.62438167043969, 41.88637740620821)",0
3,1230035,FRANKS 'N' DAWGS,FRANKS 'N' DAWGS,2094329.0,Restaurant,Risk 1 (High),1863 N CLYBOURN AVE,CHICAGO,IL,60614.0,2012-07-10,Canvass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.914990,-87.654994,"(-87.65499361162448, 41.91498953039437)",0
4,1228186,SOUTH COAST,SOUTH COAST SUSHI,1817424.0,Restaurant,Risk 1 (High),1700 S MICHIGAN AVE,CHICAGO,IL,60616.0,2013-09-20,Canvass,NaN,41.858996,-87.624106,"(-87.62410566978502, 41.85899630014676)",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2316057,LITTLE GENIUS COMMUNITY DAYCARE 11,LITTLE GENIUS COMMUNITY DAYCARE,2359451.0,Daycare (2 - 6 Years),Risk 1 (High),1000 W 103RD ST,CHICAGO,IL,60643.0,2019-10-18,License,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.706982,-87.647758,"(-87.64775773310967, 41.706982259265786)",0
59996,1170444,A J FOOD & LIQUOR INC.,FERAS FOOD & LIQUOR,2157174.0,Grocery Store,Risk 3 (Low),4265 W CERMAK RD,CHICAGO,IL,60623.0,2012-08-30,License Re-Inspection,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.851324,-87.732192,"(-87.73219217780218, 41.85132422344611)",0
59997,1098317,PITCHFORK FOOD & SALOON,PITCHFORK FOOD & SALOON,1271831.0,Restaurant,Risk 1 (High),2922-2924 W IRVING PARK RD,CHICAGO,IL,60618.0,2012-04-03,Short Form Complaint,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.954067,-87.701753,"(-87.70175255165984, 41.954066875501354)",0
59998,1632103,DATA RESTAURANT,DAATA DARBAR RESTAURANT,2446971.0,Restaurant,Risk 1 (High),2306 W DEVON AVE,CHICAGO,IL,60659.0,2016-02-23,License,NaN,41.997876,-87.687764,"(-87.6877635508729, 41.99787636318572)",0


In [ ]:
# 이 곳에 답안을 작성해주시길 바랍니다
target = 'Inspection Fail'
features = train.columns.drop([
                             'Inspection ID'
                            , 'DBA Name'
                            , 'Inspection Date'
                            , 'Latitude'
                            , 'Longitude'
                           ])

# p = p.reindex(columns=columns)

train = train[features]
test = test[features]

In [ ]:
train.isnull().sum()
test.isnull().sum()

AKA Name            235
License #             0
Facility Type       452
Risk                  2
Address               0
City                 11
State                 3
Zip                   4
Inspection Type       0
Violations         5336
Location             60
Inspection Fail       0
dtype: int64

In [ ]:
train.fillna(0,inplace=True)

## 1.2 EDA의 결과를 토대로 Feature Engineering 및 Preprocessing을 진행하세요
> 새로운 feature를 만드는 작업뿐만이 아니라, 필요한 feature가 적절한 데이터 타입을 가지고 있지 않다면 변환합니다

In [ ]:
# 이 곳에 답안을 작성해주시길 바랍니다
train.dtypes

AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Type     object
Violations          object
Location            object
Inspection Fail      int64
dtype: object

# Part 2: 모델링 (Modeling)

## 2.1 검증 방식 (Cross-validation / Hold-out validation)을 정한 후 데이터셋을 목적에 맞게 분할하세요

In [ ]:
# 이 곳에 답안을 작성해주시길 바랍니다
import numpy as np
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test.drop(columns=target)
y_test = test[target]

In [ ]:
X_train.shape, y_train.shape

((38400, 11), (38400,))

## 2.2 모델 학습을 위한 파이프라인을 구축 후 학습(fit)까지 진행하세요
> 모델은 scikit-learn, xgboost 등 어떤 라이브러리를 사용하셔도 괜찮지만 특정 라이브러리는 설치 및 설정에 시간이 소요되는 점을 감안하시기 바랍니다

In [ ]:
# 이 곳에 답안을 작성해주시길 바랍니다
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pipe = make_pipeline(
  OrdinalEncoder(), 
  SimpleImputer(strategy='constant',fill_value=0), 
  RandomForestClassifier(n_estimators=1000, random_state=10, max_depth=None)
)
pipe.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
y_pred = pipe.predict(X_val)
print('검증 정확도: ', accuracy_score(y_val, y_pred))

검증 정확도:  0.8038333333333333


## 2.3 테스트셋의 ROC / AUC 검증 점수를 예측합니다
> 제작한 모델을 활용해서 테스트셋의 **확률**을 예측하세요 (`ROC/AUC 검증 점수`를 **0.65 혹은 그 이상** 달성한다면 매우 훌륭한 모델입니다)

In [ ]:
# 이 곳에 답안을 작성해주시길 바랍니다
from sklearn.metrics import roc_auc_score

y_pred = pipe.predict(X_test)
y_pred_proba = pipe.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_pred_proba)
print('roc_auc_score: ', auc_score)

roc_auc_score:  0.5909666101155161


## 2.4 하이퍼 파라미터 튜닝을 통해서 모델을 개선하세요
> `RandomSearchCV`, `GridSearchCV` 등을 활용해서 모델의 성능을 개선합니다. 범위 설정에 따라 시간이 매우 소요될 수 있습니다.

In [ ]:
# 이 곳에 답안을 작성해주시길 바랍니다
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

pipe = make_pipeline(
  OrdinalEncoder(), 
  SimpleImputer(strategy='constant',fill_value=0), 
  RandomForestClassifier(random_state=10, max_depth=None)
)
dists = {
    'randomforestclassifier__n_estimators': randint(50, 1000), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1) # max_features
}
clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=30, 
    cv=2, 
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train)

Fitting 2 folds for each of 30 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 20.8min finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=0,
   

In [ ]:
print('최적 하이퍼파라미터: ', clf.best_params_)

최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 5, 'randomforestclassifier__max_features': 0.12265058338326074, 'randomforestclassifier__n_estimators': 294}


In [ ]:
pipe = clf.best_estimator_
y_pred = pipe.predict(X_val)
print('검증 정확도: ', accuracy_score(y_val, y_pred))

검증 정확도:  0.80475


In [ ]:
y_pred = pipe.predict(X_test)
y_pred_proba = pipe.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_pred_proba)
print('roc_auc_score: ', auc_score)

ValueError: ignored

## Part 3: 시각화 (Visualization)

> 모델의 해석을 위한 시각화를 해주세요. 아래의 제시 된 종류 중 2가지를 선택하세요 (시각화에 가장 중요한 것은 **여러분의 분석**입니다.):
> - Permutation Importances
> - Partial Dependence Plot, 1 feature isolation
> - Partial Dependence Plot, 2 features interaction
> - Shapley Values

In [ ]:
#이 곳에 답안을 작성해주시길 바랍니다
#Permutation Importance
import eli5
from eli5.sklearn import PermutationImportance

e_pipe = make_pipeline(OrdinalEncoder(), SimpleImputer(strategy='constant',fill_value=0))
model = RandomForestClassifier(n_estimators=294, max_depth=5, max_features=0.12265058338326074, random_state=10, n_jobs=-1)

encoder = e_pipe.named_steps['ordinalencoder']
X_encoded = encoder.fit_transform(X_train)

model.fit(X_encoded,y_train)
# permuter 정의
permuter = PermutationImportance(
    model, # model
    scoring='accuracy', # metric
    n_iter=5, # 다른 random seed를 사용하여 5번 반복
    random_state=10
)
# permuter 계산은 preprocessing 된 X_val을 사용합니다.
X_val_transformed = e_pipe.fit_transform(X_val)

# 실제로 fit 의미보다는 스코어를 다시 계산하는 작업입니다
permuter.fit(X_val_transformed, y_val)

feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

AKA Name           0.0
License #          0.0
Facility Type      0.0
Risk               0.0
Address            0.0
City               0.0
State              0.0
Zip                0.0
Inspection Type    0.0
Violations         0.0
Location           0.0
dtype: float64

In [ ]:
# import shap

# explainer = shap.TreeExplainer(pipe)
# shap.initjs()

Exception: ignored

In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_plot
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 144

In [ ]:
pipe = make_pipeline(
  OrdinalEncoder(), 
  SimpleImputer(strategy='constant',fill_value=0), 
  RandomForestClassifier(n_estimators=294, max_depth=5, max_features=0.12265058338326074, random_state=10, n_jobs=-1)
)

In [ ]:
# isolated = pdp_isolate(
#     model=pipe,
#     dataset=X_train, 
#     model_features=X_train.columns,
#     feature='City',
#     grid_type='percentile', # default='percentile', or 'equal'
#     num_grid_points=10 # default=10
#     )
# pdp_plot(isolated, feature_name='City');

TypeError: ignored